<a href="https://colab.research.google.com/github/ajaysuseel/MiniProject_AD/blob/main/Project_app_with_explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# PyG & CUDA‑compatible GNN libs
!pip install -q pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv \
    -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

# Core ML & NLP packages
!pip install -q torch-geometric spacy pytorch-lightning sentence-transformers transformers

# # Streamlit + ngrok for Colab hosting
# !pip install -q streamlit pyngrok

# Download spaCy English model
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66

In [3]:

import os
import json
import torch
import networkx as nx
import torch.nn as nn
import spacy
import pytorch_lightning as pl
from PIL import Image
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from transformers import BlipProcessor, BlipForConditionalGeneration
from sentence_transformers import SentenceTransformer, util
from torch_geometric.nn import SAGEConv
from tqdm.notebook import tqdm

import pickle

# ----------------------------
# 1. Config & Device
# ----------------------------
DEVICE            = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# st.set_page_config(page_title="Hazard Detection", layout="wide")

# ----------------------------
# 2. Paths (adjust as needed)
# ----------------------------
DRIVE_ROOT        = '/content/drive'
BLIP_CKPT_PATH    = os.path.join(DRIVE_ROOT, 'MyDrive/gemini_models/blip_checkpoints_17_04/blip-epoch=02-val_loss=0.0536.ckpt')
GNN_WEIGHTS_PATH  = os.path.join(DRIVE_ROOT, 'MyDrive/gemini_models/kg_models2/checkpoints/best_model.pth')
KG_PATH           = os.path.join(DRIVE_ROOT, 'MyDrive/gemini_models/kg_models2/kg_graph_aug.gpickle')

# ----------------------------
# 3. BLIP Lightning Module
# ----------------------------
class BlipLightning(pl.LightningModule):
    def __init__(self, model_name="Salesforce/blip-image-captioning-base", learning_rate=5e-5, freeze_vision=True, freeze_layers=6):
        super().__init__()
        self.model = BlipForConditionalGeneration.from_pretrained(model_name)
        self.processor = BlipProcessor.from_pretrained(model_name, use_fast=True)
        self.learning_rate = learning_rate

        if freeze_vision:
            # Freeze vision embedding layers
            for name, param in self.model.named_parameters():
                if "vision_model.embeddings" in name:
                    param.requires_grad = False
            # Freeze early vision encoder layers
            for name, param in self.model.named_parameters():
                if "vision_model.encoder.layers" in name:
                    parts = name.split(".")
                    try:
                        layer_index = int(parts[3])
                    except (IndexError, ValueError):
                        layer_index = None
                    if layer_index is not None and layer_index < freeze_layers:
                        param.requires_grad = False

    def forward(self, pixel_values, input_ids=None, attention_mask=None, labels=None):
        return self.model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

def load_blip(checkpoint_path):
    lit = BlipLightning.load_from_checkpoint(checkpoint_path)
    lit.to(DEVICE).eval()
    return lit, lit.processor

# Load BLIP once
blip, blip_processor = load_blip(BLIP_CKPT_PATH)

# ----------------------------
# 4. Load KG & Prepare Graph
# ----------------------------
with open(KG_PATH, "rb") as f:
    G = pickle.load(f)

node_to_idx = {n:i for i,n in enumerate(G.nodes())}
deg = dict(G.degree()); max_deg = max(deg.values()) or 1
edge_index = torch.tensor([[node_to_idx[u] for u,v in G.edges()],
                           [node_to_idx[v] for u,v in G.edges()]], dtype=torch.long)
x_feat = torch.tensor([[deg[n]/max_deg] for n in G.nodes()], dtype=torch.float)
data_graph = Data(x=x_feat, edge_index=edge_index).to(DEVICE)

# ----------------------------
# 5. NLP & Embedding Setup
# ----------------------------
nlp = spacy.load('en_core_web_sm')
embedder = SentenceTransformer('all-MiniLM-L6-v2')

hazard_keywords = {
    "low": [
        ("clear road", 1.0), ("dry road", 1.0), ("daytime", 1.0), ("straight road", 1.0),
        ("good visibility", 1.0), ("no traffic", 1.0), ("open road", 1.0), ("well-lit", 1.0),
        ("sunny", 1.0), ("flat terrain", 1.0), ("light traffic", 1.0), ("wide road", 1.0)
    ],
    "medium": [
        ("moderate visibility", 1.5), ("residential area", 1.5), ("curved road", 1.5),
        ("slightly wet", 1.5), ("light rain", 1.5), ("children nearby", 1.5),
        ("cyclist", 1.5), ("school zone", 1.5), ("urban traffic", 1.5),
        ("construction zone", 1.6), ("intersections", 1.6), ("speed bumps", 1.5)
    ],
    "high": [
        ("poor visibility", 2.0), ("fog", 2.0), ("heavy rain", 2.1), ("pedestrian ahead", 2.2),
        ("jaywalking", 2.2), ("nighttime", 2.0), ("icy road", 2.3), ("sharp turn", 2.1),
        ("blind spot", 2.1), ("narrow lane", 2.0), ("heavy traffic", 2.0),
        ("obstruction", 2.1), ("collision", 2.5), ("emergency vehicle", 2.2), ("road closed", 2.3)
    ]
}

def extract_triplets(text):
    doc = nlp(text.lower())
    triples = []
    for t in doc:
        if t.dep_ in ("amod","acomp") and t.head.pos_=="NOUN":
            triples.append((t.head.lemma_, t.lemma_))
        elif t.dep_=="attr" and t.head.pos_=="NOUN":
            triples.append((t.head.lemma_, t.lemma_))
        elif t.dep_=="nsubj" and t.head.pos_ in ("VERB","AUX"):
            triples.append((t.text, t.head.lemma_))
    return triples

def keyword_hazard_score(text):
    txt = text.lower()
    return torch.tensor([
        sum(txt.count(kw)*w for kw,w in hazard_keywords[lvl])
        for lvl in ("low","medium","high")
    ], dtype=torch.float)

def graph_context_score(text):
    nodes = {u for u,_ in extract_triplets(text)} | {v for _,v in extract_triplets(text)}
    vals = [deg[n]/max_deg for n in nodes if n in deg]
    m = float('nan') if not vals else sum(vals)/len(vals)
    return torch.tensor([m]*3, dtype=torch.float)

def semantic_score(text):
    emb = embedder.encode(text, convert_to_tensor=True)
    sims = []
    for lvl in ("low","medium","high"):
        kws = [kw for kw,_ in hazard_keywords[lvl]]
        kws_emb = embedder.encode(kws, convert_to_tensor=True)
        sims.append(util.cos_sim(emb, kws_emb).max().item())
    total = sum(sims) or 1.0
    return torch.tensor([s/total for s in sims], dtype=torch.float)

def compute_features(text):
    return torch.cat([
        keyword_hazard_score(text),
        graph_context_score(text),
        semantic_score(text)
    ]).unsqueeze(0)





/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# 6. Define and Load Model (updated to return per‐node attention weights)
class GraphSAGEClassifier(nn.Module):
    def __init__(self, in_c, hidden_c, out_c, dropout=0.3):
        super().__init__()
        self.sage1 = SAGEConv(in_c, hidden_c)
        self.sage2 = SAGEConv(hidden_c, hidden_c)
        self.sage3 = SAGEConv(hidden_c, hidden_c)
        self.attn_weights = nn.Parameter(torch.randn(hidden_c, 1))
        self.mlp = nn.Sequential(
            nn.Linear(hidden_c + 9, 64),
            nn.LayerNorm(64),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, out_c)
        )

    def forward(self, node_ids_batch, heuristics, graph):
        x, edge_index = graph.x, graph.edge_index

        # 1) Three GNN layers
        x = self.sage1(x, edge_index).relu()
        x = self.sage2(x, edge_index).relu()
        x = self.sage3(x, edge_index).relu()

        node_feats       = []
        attn_scores_list = []   # will hold one array of attention weights per sample

        for node_ids in node_ids_batch:
            # Gather the node embeddings for this sample
            embeds = x[node_ids]  # shape: (num_nodes_for_sample, hidden_c)

            # Compute raw attention scores: (num_nodes, 1)
            scores = embeds @ self.attn_weights

            # Normalize to get attention weights: (num_nodes, 1)
            attn = torch.softmax(scores, dim=0)

            # Weighted sum → single graph embedding vector of size (hidden_c,)
            node_feats.append((attn * embeds).sum(dim=0))

            # Save the attention weights for explainability:
            #    attn.squeeze(1) has shape (num_nodes,)
            attn_scores_list.append(attn.squeeze(1).cpu().detach().numpy())

        # Stack per‐sample graph vectors → (batch_size, hidden_c)
        graph_feats = torch.stack(node_feats)

        # Concatenate with heuristics features → (batch_size, hidden_c + 9)
        combined = torch.cat([graph_feats, heuristics.to(DEVICE)], dim=1)

        # Final MLP → (batch_size, out_c)
        logits = self.mlp(combined)

        # Return both logits and the raw attention weights per sample
        return logits, attn_scores_list


# Instantiate, load weights, and set to eval
gnn = GraphSAGEClassifier(in_c=1, hidden_c=128, out_c=3).to(DEVICE)
gnn.load_state_dict(torch.load(GNN_WEIGHTS_PATH, map_location=DEVICE))
gnn.eval()


GraphSAGEClassifier(
  (sage1): SAGEConv(1, 128, aggr=mean)
  (sage2): SAGEConv(128, 128, aggr=mean)
  (sage3): SAGEConv(128, 128, aggr=mean)
  (mlp): Sequential(
    (0): Linear(in_features=137, out_features=64, bias=True)
    (1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=64, out_features=3, bias=True)
  )
)

In [5]:
!pip install -q gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.7 MB/s eta 0:00:00


In [22]:
import gradio as gr
from PIL import Image, ImageDraw, ImageFont
import torch
import matplotlib.pyplot as plt

# —— plotting helper ——
def plot_node_attention_per_image(result, top_k=10):
    node_attn_sorted = sorted(
        result["node_attention"], key=lambda x: x["weight"], reverse=True
    )[:top_k]
    nodes   = [e["node"] for e in node_attn_sorted]
    weights = [e["weight"] for e in node_attn_sorted]

    fig, ax = plt.subplots(figsize=(6, 4))
    ax.barh(nodes, weights)
    ax.invert_yaxis()
    ax.set_xlabel("Attention Weight")
    ax.set_title("Top-10 Node Attention")
    plt.tight_layout()
    return fig

def hazard_detection_app(image_pil):
    # 1. BLIP captioning
    img = image_pil.convert("RGB")
    inputs = blip_processor(images=img, return_tensors="pt").to(DEVICE)
    out_ids = blip.model.generate(**inputs, max_new_tokens=50)
    caption = blip_processor.decode(out_ids[0], skip_special_tokens=True)

    # 2. Extract features & triplets → node indices
    feat = compute_features(caption)
    trip = extract_triplets(caption)
    nodes_idx = [
        node_to_idx[n]
        for u, v, *rest in trip
        for n in (u, v)
        if n in node_to_idx
    ]
    if not nodes_idx:
        nodes_idx = [0]

    # 3. GNN forward → logits + per-node attention
    with torch.no_grad():
        logits, attn_scores_list = gnn([nodes_idx], feat, data_graph)
    label_idx = logits.argmax(dim=1).item()
    rating = ["LOW", "MEDIUM", "HIGH"][label_idx]

    # 4. Build node_attention list
    node_attention = [
        {"node": n, "weight": w}
        for (u, v, *_), w in zip(trip, attn_scores_list[0])
        for n in (u, v)
        if n in node_to_idx
    ]

    # 5. Build attention plot
    result = {
        "filename": "<upload>",
        "caption": caption,
        "predicted_rating": rating,
        "node_attention": node_attention
    }
    fig = plot_node_attention_per_image(result, top_k=10)

    # 6. Return caption, hazard, then plot (no output image)
    return caption, rating, fig

# —— Gradio UI layout ——
custom_theme = gr.themes.Base(font=["Arial","sans-serif"]).set(
    body_text_size="16px"
)

with gr.Blocks(theme=custom_theme, css="""
    .gradio-container { max-width: 1200px; margin: auto; }
""") as demo:
    gr.Markdown("## 🚧 Single‐Image Hazard Detection")

    with gr.Row():
        # LEFT half: image uploader + run button
        with gr.Column(scale=1):
            inp_img = gr.Image(type="pil", label="📷 Upload Road Scene")
            btn_run = gr.Button("Run Inference")
        # RIGHT half: caption, hazard, then attention plot
        with gr.Column(scale=1):
            out_caption = gr.Textbox(label="📝 BLIP Caption", lines=3)
            out_hazard  = gr.Textbox(label="⚠️ Predicted Hazard Level", lines=1)
            out_plot    = gr.Plot(label="🔍 Top-10 Node Attention")

    btn_run.click(
        fn=hazard_detection_app,
        inputs=[inp_img],
        outputs=[out_caption, out_hazard, out_plot]
    )

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2e8ed4a2aa919dea54.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# import json

# def clean_notebook(path_in, path_out=None):
#  # Load the notebook as JSON
#  with open(path_in, 'r', encoding='utf-8') as f:
#      nb = json.load(f)

#  # If there's a top-level "widgets" key under metadata, remove it entirely
#  if 'widgets' in nb.get('metadata', {}):
#      print(f"→ Removing metadata.widgets from {path_in}")
#      del nb['metadata']['widgets']

#  # Write back to the same file (or to a new one if you supply path_out)
#  out_path = path_out or path_in
#  with open(out_path, 'w', encoding='utf-8') as f:
#      json.dump(nb, f, indent=1)
#  print(f"✔ Cleaned notebook saved to {out_path}")

# if __name__ == "__main__":
#  # Change this to wherever your notebook lives:
#  notebook_path = 'Project_app.ipynb'
#  clean_notebook(notebook_path)


FileNotFoundError: [Errno 2] No such file or directory: 'Project_app.ipynb'